In [ ]:
%matplotlib inline

import einops
import librosa
import matplotlib.pyplot as plt
import numpy as np
import soundfile
import torch
from IPython.display import Audio
from torch.nn import functional as F
from torchaudio.transforms import Loudness as NewLoudness

In [ ]:
import myddsp.constants as C
import myddsp.preprocessors as pre

In [ ]:
y, _ = librosa.load("../data/test/samples/violin-1.wav", sr=48000, mono=False)

In [ ]:
Audio(y, rate=48000)

In [ ]:
ld = pre.Loudness()

In [ ]:
yy = torch.from_numpy(y)
yy = yy.unsqueeze(0)
amps = ld(yy).flatten().numpy()

In [ ]:
plt.plot(amps)

In [ ]:
amps.shape

In [ ]:
def randomize_phase(x):
    s = torch.fft.rfft(x)
    s_ = torch.polar(s.abs(), torch.rand_like(s.angle()) * 2.0 * np.pi)
    x_ = torch.fft.irfft(s_)

    return x_

In [ ]:
shit = randomize_phase(yy)

In [ ]:
yy.shape

In [ ]:
amps1 = ld(yy).flatten().numpy()

In [ ]:
all(np.isclose(amps, amps1))

In [ ]:
rf = librosa.util.frame(y, frame_length=C.N_FFT, hop_length=C.HOP_LENGTH, writeable=True)
rf = torch.from_numpy(rf)

In [ ]:
tl = NewLoudness(C.SAMPLE_RATE)

In [ ]:
rf.shape

In [ ]:
c, n, f = rf.shape
rf_ = einops.rearrange(rf, "c n f -> f c n")
win = torch.hann_window(n)
rf_ = einops.einsum(rf_, win, "f c n, n -> f c n")
start = (19200 - n) // 2
end = (19200 - n) - start
rf_ = F.pad(rf_, (start, end))
amp2 = tl(rf_)

In [ ]:
plt.plot(amp2.numpy())

In [ ]:
filtered = amp2[~torch.isnan(amp2)]

In [ ]:
plt.plot(filtered.numpy())

In [ ]:
filtered2 = amps[amps >= -70]

In [ ]:
plt.plot(filtered2)